# Steps to prepare yield tables for NuPyCEE

How to create your own yield tables and modify existing tables. Current yield tables
are available in the yield_tables directory. There are two formats of yield tables to be
used.

The script read_yields.py is used by SYGMA and OMEGA and allows to deal with yield tables.

In [1]:
%matplotlib nbagg
import read_yields as ry
import sygma as s

## I. Write your own yield tables for AGB and massive stars

Tables for each stellar model need to have the same number of isotopes and the same table header structure!!
Also each table header must contain the Table identifier (M=1.0,Z=0.02), the lifetime of the star and the final remnant/WD mass. Here an example:

H Table: (M=1.0,Z=0.02) 

H Lifetime: 1.215E+10

H Mfinal: 5.601E-01

Have a look at in yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12delay.txt for an example.

Below we write some dummy values in a yield table:

In [19]:
Zs = [0.02,0.01] #Metallicity         
Ms = [[1,5,15],[1,5,15]] #Initial masses
# 4 models in total, each of them the same number of isotopes!
isos = ['H-1','He-4','C-12']
yields_star = [0.1,0.2,0.01] #Msun
data_star = [yields_star] #provide only yields 
# structure: [Zs[0],Zs[1]] where Z[0] => Ms[0] and Z[1] => Ms[1]
yields_all = [len(Ms[0])*[data_star],len(Ms[1])*[data_star]]    

In [20]:
#define header data, define always all fields.
table_name = 'Yield table'
units='Msun, year'          

In [21]:
#additional necessary data per stellar model
col_attrs = ['Lifetime', 'Mfinal']
data_star =[2e9,0.9] #as specified with Units, yr and Msun
col_attrs_data = [len(Ms[0])*[data_star],len(Ms[1])*[data_star]]   

In [22]:
#data columns, 'Yields' required but other data can be added (in data_star).
data_cols = ['Yields']

In [23]:
ry.write_tables(data=yields_all,data_cols=data_cols,
                Zs=Zs,Ms=Ms,isos=isos,
                col_attrs=col_attrs,col_attrs_data=col_attrs_data,
                units=units,table_name=table_name,filename='isotope_yield_table_mod.txt')

Yields table  isotope_yield_table_mod.txt  created.


#### Test: Read the table

In [24]:
new_table_name='isotope_yield_table_mod.txt'

In [25]:
table=ry.read_nugrid_yields(new_table_name)

In [29]:
#try tetting H-1 you wrote
print table.metallicities
print table.get(Z=0.02,quantity='masses')
print table.get(M=1,Z=0.02,specie='C-12')

[0.02, 0.01]
[1.0, 5.0, 15.0]
0.01


In [36]:
#You can try to run SYGMA with with the yield table for a test
s1=s.sygma(iniZ=0.02,table='Teaching/'+new_table_name)

/Users/christian/anaconda/lib/python2.7/site-packages/numpy/lib/polynomial.py:594: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)


SYGMA run in progress..
   SYGMA run completed - Run time: 0.25s


This example assumes that isotopes available in table1 are also in table2. If not, the loops below have
to be adjusted accordingly.

### Example: Merging two yield tables:

In [37]:
table1=ry.read_nugrid_yields('../yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12delay.txt')

In [38]:
table2=ry.read_nugrid_yields('../yield_tables/agb_and_massive_stars_nugrid_MESAonly_fryer12rapid.txt')

In [39]:
print table1.metallicities
print table1.data_cols
Zs = table1.metallicities
for Z in Zs:
    Ms=table1.get(Z=Z,quantity='masses')
    for M in Ms:
        isos = table1.get(M=M,Z=Z,quantity='Isotopes')
        for iso in isos:
            y_iso1 = table1.get(M=M,Z=Z,specie=iso)
            y_iso2 = table2.get(M=M,Z=Z,specie=iso)
            y_mix = (y_iso1+y_iso2)/2.
            #The set function only modifies an existing entry
            table1.set(M=M,Z=Z,specie=iso,value=y_mix)
            

[0.02, 0.01, 0.006, 0.001, 0.0001]
['Isotopes', 'Yields', 'X0', 'Z', 'A']


In [40]:
table1.write_table(filename='agb_and_massive_stars_test_merge.txt')

Yields table  agb_and_massive_stars_test_merge.txt  created.


## II. Write your own yield tables for other sources

The second table format used usually makes up just one table and can
be easily written with custom scripts.